In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [13]:
# read in the data
data = pd.read_csv('data_2018_hlthmntl.csv')

In [14]:
fname = '2018_all_marital_PS.json'
with open(fname, 'r') as infile:
    param_dict = json.load(infile)

param_dict

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 'log2',
 'min_samples_leaf': 6,
 'min_samples_split': 2,
 'n_estimators': 400,
 'test_score': 0.5909,
 'train_score': 0.6569}

In [15]:
from sklearn.ensemble import RandomForestClassifier
best_rf = RandomForestClassifier(bootstrap=param_dict['bootstrap'], 
                                 max_depth=param_dict['max_depth'], 
                                 max_features=param_dict['max_features'],
                                 n_estimators=param_dict['n_estimators'],
                                 min_samples_leaf=param_dict['min_samples_leaf'],
                                 min_samples_split=param_dict['min_samples_split'],
                                 random_state=99)

In [16]:
best_rf.fit(data.drop(columns=['hlthmntl', 'marital']), y=data['marital'])

RandomForestClassifier(max_depth=20, max_features='log2', min_samples_leaf=6,
                       n_estimators=400, random_state=99)

In [17]:
data['propensity'] = best_rf.predict_proba(data.drop(columns=['hlthmntl', 'marital']))[:,1]
data.shape

(2296, 10)

In [18]:
# remove propensity score that is 1 or zero
data = data.loc[~data['propensity'].isin([1,0])]
data.shape

(2296, 10)

In [19]:
def ipw_cal(propensity_score, marital_status):
    '''
    Calculates IPW score for given propensity score and marital status
    :param propensity_score: propensity score
    :param marital_status: marital status
    :return: get the inverse propensity score weights
    '''
    if marital_status == 1:
        weighting = 1/propensity_score

    if marital_status == 0:
        weighting = 1/(1-propensity_score)

    return weighting

In [20]:
data['weighting'] = [ipw_cal(x, y) for x, y in zip(data['propensity'], data['marital'])]

In [21]:
# outcome model
import statsmodels.api as sm
Y = np.array(data['hlthmntl'],dtype=float)
X = np.array(data.drop(columns=['hlthmntl', 'propensity', 'weighting']),dtype=float)
weights = np.array(data['weighting'],dtype=float)
wls_model = sm.WLS(Y,X, weights=weights)
wls_model.exog_names[:] = list(data.drop(columns=['hlthmntl', 'propensity', 'weighting']).columns)
results = wls_model.fit()
results.params

array([-0.16070375, -0.11773627,  0.23721756,  0.07083836,  0.06011329,
        1.90106791,  1.8410952 ,  1.96635294])

In [22]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.086
Model:                            WLS   Adj. R-squared:                  0.083
Method:                 Least Squares   F-statistic:                     30.81
Date:                Sat, 11 May 2024   Prob (F-statistic):           5.21e-41
Time:                        15:41:25   Log-Likelihood:                -3129.6
No. Observations:                2296   AIC:                             6275.
Df Residuals:                    2288   BIC:                             6321.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
marital          -0.1607      0.039     -4.174      0.000      -0.236      -0.085
degree           -0.1177      0.017     -7.095      0.000      -0.150      -0.085
satfin            0.2372      0.027      8.788      0.000       0.184       0.290
neisafe           0.0708      0.030      2.339      0.019       0.011       0.130
relpersn          0.0601      0.019      3.112      0.002       0.022       0.098
race_is_white     1.9011      0.095     20.096      0.000       1.716       2.087
race_is_black     1.8411      0.108     17.045      0.000       1.629       2.053
race_is_other     1.9664      0.111     17.703      0.000       1.749       2.184
==============================================================================
Omnibus:                       45.993   Durbin-Watson:                   2.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               48.265
Skew:                           0.350   Prob(JB):                     3.31e-11
Kurtosis:                       3.120   Cond. No.                         36.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""